# Agents Creation and Basic Concepts

On PydanticAI we use the class `Agent` to create an agent, where we can specify:

| Component | Description |
| --- | --- |
| Model to be used | For example `openai:gpt-4o` or `google:gemnini-2.5-flash` |
| System Prompt | Initial message that orients the LLM behavior |
| Type of dependencies | If the agent needs external data |
| Type of response | the Pydantic model that defines the response |

In [2]:
import os
from dotenv import load_dotenv

# loads dotenv
load_dotenv()

from pydantic_ai import Agent

model = 'anthropic:claude-sonnet-4-5'

prompt = """"
<persona>
You are a helpful Financial Historian, you can answer questions about the history of finance.
</persona>
<background>
You've made your major in History at Harvard University. With PhD in Financial History.
</background>
<goals>
- Your goal is to answer questions about the history of finance.
- You should use the background information to answer the questions.
- You should be friendly and engaging.
</goals>

<guidelines>
- You should use the background information to answer the questions.
- You should be friendly and engaging.
- You should NEVER make up information, you should only answer with the information provided in the background.
- You should NEVER answer anything that is not related to the history of finance.
- When prompted with a question that is not related to the history of finance, you should answer "That's with the other teacher".
</guidelines>
"""

agent = Agent(
    model=model,
    system_prompt=prompt,
)

async def make_question(question: str) -> str:
    response = await agent.run(question)
    return response.output

question = "What is the history of the stock market?"

result = await make_question(question)
print(result)

# The History of the Stock Market

Great question! The stock market has a fascinating evolution spanning several centuries.

## Early Beginnings (1600s)

The modern stock market traces its roots to **Amsterdam in 1602**, when the Dutch East India Company issued the first paper shares. This allowed the company to raise capital for its expensive voyages to Asia, and investors could buy and sell these shares on the Amsterdam Stock Exchange - the world's first official stock exchange.

## London and Early Markets (1700s)

The **London Stock Exchange** emerged in the early 1700s, initially operating out of coffeehouses where traders would meet. By 1773, it had a more formal structure with regulated trading.

## American Stock Markets (Late 1700s-1800s)

In the United States:
- The **Philadelphia Stock Exchange** was established in 1790
- The **New York Stock Exchange (NYSE)** was founded in 1792 with the famous Buttonwood Agreement, where 24 stockbrokers met under a buttonwood tree on Wall 

## System Prompts: Dynamics and Statics

Static Prompts: Passed through the creation of the agent

Dynamic Prompts: Defined by functions decorated by `@agent.system_prompt` that can access the execution context (RunContext)

In [3]:
from pydantic_ai import Agent, RunContext

agent = Agent(
    model=model,
    deps_type=str, # Here we specify the type of the dependency (dynamic variable)
    system_prompt=prompt,
)

@agent.system_prompt
def obtain_users_name(context: RunContext) -> str:
    return f"The name of the user is {context.deps}."

async def make_question_with_name(question: str) -> str:
    name = "Pedro"
    response = await agent.run(question, deps=name)
    return response.output

question = "What is the history of the stock market?"

result = await make_question_with_name(question)
print(result)

Hello Pedro! Great question! I'd be happy to walk you through the fascinating history of the stock market.

## Early Beginnings

The concept of stock markets has roots going back centuries. The first real stock exchange is generally considered to be the **Amsterdam Stock Exchange**, founded in 1602 by the Dutch East India Company (VOC). This was revolutionary - the VOC issued shares that could be traded among investors, creating the first publicly traded company and a marketplace for those shares.

## Development in Europe

Throughout the 1600s and 1700s, stock trading expanded across Europe. The **London Stock Exchange** emerged from coffeehouses where traders met, officially forming in 1801 (though trading in London dates back much earlier).

## American Stock Markets

In the United States, the stock market began informally. The **Buttonwood Agreement** of 1792 - signed by 24 stockbrokers under a buttonwood tree on Wall Street - laid the foundation for what would become the **New Yor

## Models from Pydantic
PydanticAI allows us to define well structured outputs/inputs for our Agent calls

In [ ]:
from pydantic import BaseModel, Field

class ChatResponse(BaseModel):
    answer: str = Field(description="The answer to the user's question")
    source: str = Field(description="The source used to answer the question, should be a website.")

class UserInformation(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")

agent = Agent(
    model=model, # Infered from the ones we used on the previous cells
    output_type=ChatResponse, # Here we specify the output type
    system_prompt=prompt,
    deps_type=UserInformation, # Here we specify the type of the dependency (dynamic variable)
)

@agent.system_prompt
def obtain_user_information(context: RunContext) -> str:
    return f"The name of the user is {context.deps.name} and the age is {context.deps.age}."

async def make_question_with_user_information(question: str) -> str:
    name = "Pedro"
    age = 24
    response = await agent.run(question, deps=UserInformation(name=name, age=age))
    return response.output

result = await make_question_with_user_information("Can you help me understand the history of Bitcoin for someone my age?")
print(result)